<a href="https://colab.research.google.com/github/leolaipelt/EarthEnginePythonBasico/blob/master/Iniciando_com_EarthEngine_Pyhton.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Vamos iniciar com Geoprocessamento em Python utilizando EarthEngine!

### 1º - Instalar a biblioteca utilizando !pip install earthengine-api
### 2º - Autenticar a tua permissão para utilizar o programa 

    import ee
    ee.Authenticate()
### 3º Passo Vamos ver se funcionou!

In [0]:
!pip install earthengine-api

In [0]:
import ee

ee.Authenticate()

In [5]:
#Inicializando a Biblioteca EE

ee.Initialize()

# Descobrindo a Elevação do IPH
dem = ee.Image('USGS/SRTMGL1_003')
xy = ee.Geometry.Point([-51.113900,-30.076286])  #Longitude #Latitude
elev = dem.sample(xy, 30).first().get('elevation').getInfo()
print('O IPH esta em uma altura de (m):', elev)

O IPH esta em uma altura de (m): 82


In [0]:
##Mas nós queremos também visualizar de alguma forma as nossas imagens!
#O EarthEngine Javascript já vem com um visualizador construido, mas no caso do Python nós precisamos elaborar um! 
#Mas é fácil!

#1º Vamos baixar uma nova biblioteca, a Folium! 

## Biblioteca Folium

Folium é utilizado para interagir com mapas e interagir com rasters ou shapes através de páginas web. Para isso, ela foi desenvolvida a partir da bliblioteca Leaflet.js (Javascript) que desenvolveu esse tipo de visualização de mapas em página web!

#### Link para acessar a página da Folium

https://python-visualization.github.io/folium/

### Página Leaflet.js (apenas por curiosidade)

https://leafletjs.com/

In [0]:
!pip install folium

In [0]:
#Agora vamos escrever uma função (def) para que o Folium possa mostrar uma imagem do earthengine no mapa!

def add_ee_layer(self,imagem,visParams, nome):
  mapID = ee.Image(imagem).getMapId(visParams)
  folium.raster_layers.TileLayer(
    tiles ="https://earthengine.googleapis.com/map/"+mapID['mapid']+
      "/{z}/{x}/{y}?token="+mapID['token'],
    attr = "Nosso Mapinha do Curso de Python",
    name = nome,
    overlay = True,
    control = True
  ).add_to(self)

In [0]:
import folium

#Agora vamos atribuir a um mapa folum
folium.Map.add_ee_layer = add_ee_layer

#Vamos pegar agora uma imagem do earthengine qualquer.
dem = ee.Image('USGS/SRTMGL1_003')

#Toda imagem precisa ter algum parametro de visualizaçao, esses parametros serao atribuidos em visParams

visParams = {'min':0, 'max':3000}
#Como escolhemos cores? Simples. Em visParams adicionamos apos min e max : 'palette':['225ea8','41b6c4','a1dab4','ffffcc']
#visParams = {'min':0, 'max':3000,'palette':['225ea8','41b6c4','a1dab4','ffffcc']}

# Criando um mapa folium
meuMap = folium.Map(location=[20, 0], zoom_start=3, height=500)

# Adicionando no mapa a imagem que criamos
meuMap.add_ee_layer(dem, visParams, 'DEM')

# Adicionando um controlador de layers no nosso mapa
meuMap.add_child(folium.LayerControl())

#Ver nosso mapa!
display(meuMap)

#### Ta mas eu não quero o Mundo Todo! Quero saber só do Brasil mesmo. Como eu faço?
#### Temos duas opções:
* Inserir um shapefile do meu interesse no assets do google earthengine
* Utilizar os "shapefiles" que temos disponíveis no datasets do earthengine!



In [0]:
#Vamos utilizar um dataset já disponível!

FakeShapefilePaises = (ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')
      .filterMetadata('country_na', 'equals', 'Brazil'))


In [0]:
Brazil_DEM=dem.clip(FakeShapefilePaises)

In [0]:
meuMap = folium.Map(location=[20, 0], zoom_start=3, height=500)

# Adicionando no mapa a imagem que criamos
meuMap.add_ee_layer(Brazil_DEM, visParams, 'DEM_BRASIL')
# Adicionando um controlador de layers no nosso mapa
meuMap.add_child(folium.LayerControl())
#Ver nosso mapa!
display(meuMap)

### Ah não sei. Isso ai eu fazia baixando no LabGeo as informacoes e eu nao precisava programar. Odeio Programar.
#### Ok. Vamos dar mais um passo no nosso conhecimento de EarthEngine
#Vamos mexer com ImageCollections!

O que é um Image Collection? É uma biblioteca com diversas imagens! Ou seja:

### Uma imagem é um "image" no EE, que pode ter diversas bandas (B1,B2,B3,B4).

Uma coleção de imagens "ImageCollection" pode possuir diversas imagens (variando por ano,mes,dia,horas..) e cada uma dessas imagens podem ter diversas bandas!!

Vamos mexer então com a ImageCollection da Landsat!

A ideia é Calcular o NDVI medio para um ano inteiro. Para TODAS as imagens que o Landsat produziu para o Brasil!

In [0]:
#Primeira coisa, vamos chamar uma biblioteca do Landsat

L8SR= ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')

#Vamos Escolher as Datas que nos interessa e depois faze a media delas com o comando mean()

Image = L8SR.filterDate('2018-01-01', '2018-12-31').mean()


In [0]:
#Por enquanto, nós fizemos isso para #TODAS as imagens da Coleção Tier1 de Surface Reflectance que existem do LANDSAT entre jan de 2018 a dez de 2018
#Como ja temos nosso fakeShape do Brasil, temos que realizar o corte com o comando clip!

LANDSAT_Corte=Image.clip(FakeShapefilePaises)

In [0]:
#Agora, vamos calcular o NDVI utilizando uma funcao muito legal do EE que é normalizedDifference

NDVI =  LANDSAT_Corte.normalizedDifference(['B5', 'B4']).rename('NDVI'); #Pq dividir por 10000? Porque é a contante para as bandas do LANDSAT


In [0]:
meuMap = folium.Map(location=[20, 0], zoom_start=3, height=500)

visParams={"palette": "0000cc,0066cc,00cccc,0033cc,0033cc,cc6600,cc9900,cccc00,99cc00,66cc00"}
# Adicionando no mapa a imagem que criamos
meuMap.add_ee_layer(NDVI, visParams, 'NDVI_BRASIL')
# Adicionando um controlador de layers no nosso mapa
meuMap.add_child(folium.LayerControl())
#Ver nosso mapa!
display(meuMap)

In [0]:
#E se a gente quiser ver apenas uma faixa de NDVI. Como que faz?
meuMap = folium.Map(location=[20, 0], zoom_start=3, height=500)

visParams={"palette": "0000cc,0066cc,00cccc,0033cc,0033cc,cc6600,cc9900,cccc00,99cc00,66cc00"}
#Assim
NDVIMaior = NDVI.updateMask(NDVI.gte(0.4)) #gte significa Greater than or equal # se quisermos um numero menor que algo ou igual, usamos lte (lower than or equal)

meuMap.add_ee_layer(NDVIMaior, visParams, 'NDVI_MAIOR04')
# Adicionando um controlador de layers no nosso mapa
meuMap.add_child(folium.LayerControl())
#Ver nosso mapa!
display(meuMap)

Vamos agora dar uma olhada em como podemos acessas algumas informações relevantes das Imagens.

In [9]:
#Exemplo de Imagem

image = ee.Image('LANDSAT/LC08/C01/T1/LC08_044034_20140318')

#Buscando informações das Bandas da imagem
bandNames = image.bandNames()
print(bandNames.getInfo())

# Para saber a Projeção
b1proj = image.select('B1').projection()
print( b1proj.getInfo())

#Escala (em metros)
b1scale = image.select('B1').projection().nominalScale()
print( b1scale.getInfo()) 

['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 'B10', 'B11', 'BQA']
{'type': 'Projection', 'crs': 'EPSG:32610', 'transform': [30.0, 0.0, 460785.0, 0.0, -30.0, 4264215.0]}
30.0


In [10]:
# SE a gente quiser consultar todos os Metadados podemos fazer o seguinte:
metadados = image.propertyNames()
metadados.getInfo() 

['RADIANCE_MULT_BAND_5',
 'RADIANCE_MULT_BAND_6',
 'RADIANCE_MULT_BAND_3',
 'RADIANCE_MULT_BAND_4',
 'RADIANCE_MULT_BAND_1',
 'RADIANCE_MULT_BAND_2',
 'system:id',
 'K2_CONSTANT_BAND_11',
 'K2_CONSTANT_BAND_10',
 'system:footprint',
 'REFLECTIVE_SAMPLES',
 'SUN_AZIMUTH',
 'CPF_NAME',
 'DATE_ACQUIRED',
 'ELLIPSOID',
 'google:registration_offset_x',
 'google:registration_offset_y',
 'STATION_ID',
 'RESAMPLING_OPTION',
 'ORIENTATION',
 'WRS_ROW',
 'RADIANCE_MULT_BAND_9',
 'TARGET_WRS_ROW',
 'RADIANCE_MULT_BAND_7',
 'RADIANCE_MULT_BAND_8',
 'IMAGE_QUALITY_TIRS',
 'TRUNCATION_OLI',
 'CLOUD_COVER',
 'GEOMETRIC_RMSE_VERIFY',
 'COLLECTION_CATEGORY',
 'GRID_CELL_SIZE_REFLECTIVE',
 'CLOUD_COVER_LAND',
 'GEOMETRIC_RMSE_MODEL',
 'COLLECTION_NUMBER',
 'IMAGE_QUALITY_OLI',
 'LANDSAT_SCENE_ID',
 'WRS_PATH',
 'google:registration_count',
 'PANCHROMATIC_SAMPLES',
 'PANCHROMATIC_LINES',
 'GEOMETRIC_RMSE_MODEL_Y',
 'REFLECTIVE_LINES',
 'TIRS_STRAY_LIGHT_CORRECTION_SOURCE',
 'GEOMETRIC_RMSE_MODEL_X',
 'sy

In [25]:
#Se quisermos uma informação específica do Metadados, podemos usar a função get
print("Usando a função get:")
cobertura_de_nuvens = image.get('CLOUD_COVER')
cobertura_de_nuvens.getInfo() 



Usando a função get:


0.06

In [0]:
# Load two 5-year Landsat 7 composites.
landsat1999 = ee.Image('LANDSAT/LE7_TOA_5YEAR/1999_2003')
landsat2008 = ee.Image('LANDSAT/LE7_TOA_5YEAR/2008_2012')

# Compute NDVI the hard way.
ndvi1999 = landsat1999.select('B4')\
                      .subtract(landsat1999.select('B3'))\
                      .divide(landsat1999.select('B4').add(landsat1999.select('B3')))

# Compute NDVI the easy way.
ndvi2008 = landsat2008.normalizedDifference(['B4', 'B3'])

In [0]:
# Load a Landsat 8 image.
image = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140318')

# Create NDVI and NDWI spectral indices.
ndvi = image.normalizedDifference(['B5', 'B4'])
ndwi = image.normalizedDifference(['B3', 'B5'])

# Create a binary layer using logical operations.
bare = ndvi.lt(0.2).And(ndwi.lt(0))

bare = bare.updateMask(bare)